In [42]:
from random import random, randint, seed, uniform, choice, sample
from statistics import mean
from copy import deepcopy
import matplotlib.pyplot as plt
import math

def protected_div(x, y):
    if y == 0:
        return 1
    return x / y

def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y

FUNCTIONS = [add, sub, mul, protected_div]
TERMINALS = [0, 1, 2, 3, 4, 5,6,7,8,9]


In [43]:
def create_tree(max_depth, depth=0, is_root=True):
    if depth == max_depth or (not is_root and random() > 0.5):
        return choice(TERMINALS)
    func = choice(FUNCTIONS)
    return [func, create_tree(max_depth, depth+1, False), create_tree(max_depth, depth+1, False)]


In [44]:
target_sequence = [0, 0, 1, 2, 4, 5, 7, 9, 12, 14, 17, 21, 24, 28, 32, 37, 42, 47, 52, 58, 64, 70, 77, 84, 91, 98, 105, 114, 122, 131]

def evaluate(tree):
    diff = 0
    for i, target in enumerate(target_sequence):
        result = execute_tree(tree, i)
        diff += abs(result - target)
    return diff


In [45]:
def execute_tree(tree, x):
    if not isinstance(tree, list):
        return tree if tree != 'x' else x
    func = tree[0]
    left = execute_tree(tree[1], x)
    right = execute_tree(tree[2], x)
    return func(left, right)


In [46]:
def crossover(tree1, tree2):
    if not isinstance(tree1, list) or not isinstance(tree2, list):
        return deepcopy(tree1), deepcopy(tree2)
    tree1[1], tree2[1] = tree2[1], tree1[1]  # swap subtrees
    return tree1, tree2

def mutate(tree):
    if not isinstance(tree, list):
        return create_tree(2)
    mutate_branch = randint(1, 2)
    tree[mutate_branch] = create_tree(2)
    return tree


In [47]:
population_size = 100
max_generations = 200
population = [create_tree(8) for _ in range(population_size)]

for generation in range(max_generations):
    population = sorted(population, key=evaluate)
    next_gen = population[:2]  # Elitism: Keep the best two

    while len(next_gen) < population_size:
        if random() < 0.7:  # Crossover
            ind1, ind2 = sample(population[:10], 2)  # Top 10 individuals
            off1, off2 = crossover(deepcopy(ind1), deepcopy(ind2))
            next_gen.extend([off1, off2])
        else:  # Mutation
            ind = choice(population[:10])
            next_gen.append(mutate(deepcopy(ind)))

    population = next_gen

best_individual = population[0]
print('Best Individual:', best_individual)
print('Fitness:', evaluate(best_individual))


Best Individual: [<function add at 0x000001E9A6847EC0>, [<function mul at 0x000001E9A68776A0>, 4, 8], [<function protected_div at 0x000001E9A6847F60>, 1, 5]]
Fitness: 1015.9999999999997
